In [1]:
#========================================================
#________________________________________________________
#Filename: cyber_rover_v3.5.ipynb
#Author: Sopumelela Sandekela
#Create: 23/08/2024
#Operationg System: DexterOS
#Version: v3.5 (Final Release)
#Description: This script creates memory that remembers instructions,
#When finished all the movements must be performed backwards.
#________________________________________________________

"""
__________________________________
ALGORITHM SEQUENCE
----------------------------------
1.import all the neccessary libraries.
2.Prepare/Create robot live memory, check if exists.
3.Get number of steps the user wants the robot to take.
4.Prepare Robot Movement System & Indication System.
5.Generate Random Automated Movements & Save To CSV.
6.Create Control Buttons.
7.Restore & Run Movements From Live Memory.

"""

import numpy as np
import pandas as pd
import random
import time

#__________________________________
#import Robot movements library...
#---------------------------
import easygopigo3 as easy
gopigo3 = easy.EasyGoPiGo3()
#---------------------------

#__________________________________________
#import widgets libraries for button use...
#--------------------------------------
#Import libraries for the button use...
from ipywidgets import widgets, Layout
from IPython.display import display
#--------------------------------------


#----------------------------------------------------------------------
#ERROR HANDLING: Check if live memory text file exist...
#----------------------------------------------------------------------
try:
    file = open("live_memory.txt", "r")

except FileNotFoundError:
    print("=====================================================")
    print("***-The file does not exist. Creating a new file.-***")
    print("=====================================================")
    print()
    # Create a new file
    file = open("live_memory.txt", "w")
    file.close()
#----------------------------------------------------------------------

#---------------------------------------------------
#Clear the text file before use...
#---------------------------------------------------
#Open the file in write mode to clear its contents
with open("live_memory.txt", "w") as file:
    pass #The pass statement does nothing, but the file is now empty
        
file.close()
#--------------------------------------------------- 


In [2]:
"""
==============================================================================
********************-GET NUMBER OF STEPS FROM USER-***************************
==============================================================================

"""
#_______________________________________________________________
#GET NUMBER OF STEPS THE USER WANT THE ROBOT TO TAKE...
#_______________________________________________________________
# Create an IntText widget for integer input
int_input = widgets.IntText(
    value=0,
    description='Enter steps:',
    disabled=False
)

# Display the widget
display(int_input)
#-----------------------------------------------------

IntText(value=0, description='Enter steps:')

In [3]:

""""
================================================================================================
*********************-PREPARE ROBOT MOVEMENTS SYSTEM & INDICATION SYSTEM-***********************
================================================================================================

"""

#__________________________________
#import Robot movements library...
#---------------------------
import easygopigo3 as easy
gopigo3 = easy.EasyGoPiGo3()
#---------------------------

#============================================================
#_________________________________________________

#FUNCTIONS: Prepare indicators & STOP...
#_________________________________________________

red = (255, 0, 0) #RGB color combo for red...
amber = (255, 191, 0) #RGB color combo for Amber...

def indicate_right():
    def blink_amber_left():
        gopigo3.set_left_eye_color(amber) # set led color AMBER.
        gopigo3.open_left_eye() #set led on.
        time.sleep(0.2) #sleep 0.2 seconds.
        gopigo3.close_left_eye() #set off led.

    break_lights_on() #switch on break lights.   
    blink_amber_left()#blink left indicator.
    time.sleep(0.2)
    blink_amber_left()
    time.sleep(0.2)
    blink_amber_left() #last blink left indicator.
    break_lights_off() #switch off break lights.


def indicate_left():
    def blink_amber_right():
        gopigo3.set_right_eye_color(amber)
        gopigo3.open_right_eye()
        time.sleep(0.2)
        gopigo3.close_right_eye()

    break_lights_on()
    blink_amber_right()
    time.sleep(0.2)
    blink_amber_right()
    time.sleep(0.2)
    blink_amber_right()
    break_lights_on()

def stop_indicator():
    def blink_stop_red():
        gopigo3.set_eye_color(red) #set led color RED. 
        gopigo3.led_on('left') 
        gopigo3.led_on('right') #turn on break lights.       
        gopigo3.open_eyes() #switch on red light. 
        time.sleep(0.2)
        gopigo3.led_off('left')
        gopigo3.led_off('right')        
        gopigo3.close_eyes() #switch off red light.
        
    blink_stop_red()
    
def break_lights_on():
        gopigo3.led_on('left') #turn on left red led.
        gopigo3.led_on('right')#turn on right red led.
        
def break_lights_off():
        gopigo3.led_off('left')#turn off left red led.
        gopigo3.led_off('right') #turn off right
    
#_________________________________________________              
#============================================================ 



#=============================================================================
#______________________________________________________________________
#FUNCTION: PREPARE ROBOT MOVEMENTS...
#______________________________________________________________________
def move_robot(directions, sign):

    #---------------------------------------------------
    #Clear the text file before use...
    #---------------------------------------------------
    #Open the file in write mode to clear its contents
    with open("live_memory.txt", "w") as file:
        pass #The pass statement does nothing, but the file is now empty
        
    file.close() #close text file.
    #---------------------------------------------------    

    #FUNCTION: append parameter(move) value in live memory
    def write_line(movement, sign):
        with open('live_memory.txt', 'a') as file: #open in appending mode.
            file.write(sign + movement + '\n') #append value in live_memory.
            file.close() #close text file.
    
    #If statements takes each direction and take action.
    for move in directions:
        if move == "forward":
            gopigo3.drive_cm(70) #drive forward for 70cm.
            write_line('Moving forwards', sign) #call write_line to save move in txt file.
        #-------------------------------     
        elif move == "backward":
            gopigo3.backward() #start driving backwards.
            time.sleep(5) #sleep for 5 seconds.
            gopigo3.stop() #stop driving backwards.
            write_line('Moving backwards', sign) #Save in live_memory txt.
        #------------------------------- 
        elif move == "left":
            indicate_left() #call indicate_left function before turning left.
            gopigo3.left() #Start turning left.
            time.sleep(1.1) #sleep 1.1 seconds.
            gopigo3.stop() #stop turning.
            write_line('Moving left', sign) #Save to live_memory.
        #-------------------------------     
        elif move == "right":
            indicate_right() #call indicate_right function before turning right.
            gopigo3.right() #Start turning right.
            time.sleep(1.1) #sleep 1.1 seconds.
            gopigo3.stop() #Stop turning.
            write_line('Moving right', sign) #Save to live_memory.
        #-------------------------------    
        elif move == "left U-turn":
            gopigo3.drive_cm(50) #drive forward for 50cm.
            write_line('L.U-Turn-Step1', sign)#Save step 1.
            indicate_left() #indicate left.
            gopigo3.left() #U-Turn left.
            time.sleep(2.2) #Sleep 2.2 seconds.
            gopigo3.stop #Stop turning.
            write_line('L.U-Turn-Step2', sign) #Save step 2.
            gopigo3.drive_cm(50) #drive forward for 20cm.
            write_line('L.U-Turn-Step3', sign) #Save step 3.
            indicate_left() #indicate left.
            gopigo3.left() #Turn left.
            time.sleep(2.2) #Sleep 2.2 seconds.
            gopigo3.stop() #Stop turning.
            write_line('L.U-Turn-Step4', sign) #Save step 4.
        #-------------------------------     
        elif move == "right U-turn":
            gopigo3.drive_cm(50) #drive forward for 50cm.
            write_line('R.U-Turn-Step1', sign) #Save step 1.
            indicate_right() #indicate right.
            gopigo3.right() #U-Turn right.
            time.sleep(2.2) #Sleep 2.2 seconds.
            gopigo3.stop() #Stop turning.
            write_line('R.U-Turn-Step2', sign) #Save step 2.
            gopigo3.drive_cm(50) #drive forward for 50cm.
            write_line('R.U-Turn-Step3', sign) #Save step 3.
            indicate_right() #indicate right.
            gopigo3.right() #U-Turn right.
            time.sleep(2.2) #Sleep 2.2 seconds.
            gopigo3.stop() #Stop turning.
            write_line('R.U-Turn-Step4', sign) #Save step 4. 
        #-------------------------------
        elif move == "L.U-Turn-Step1":
            gopigo3.drive_cm(50) #drive forward for 50cm.
            write_line('L.U-Turn-Step1', sign) #Save step 1.
        #-------------------------------
        elif move == "L.U-Turn-Step2":
            indicate_left() #indicate left.
            gopigo3.left() #Turn left.
            time.sleep(2.2) #Sleep 2.2 seconds.
            gopigo3.stop() #Stop turning.
            write_line('L.U-Turn-Step2', sign) #Save step 2.
        #-------------------------------
        elif move == "L.U-Turn-Step3":
            gopigo3.drive_cm(50) #drive forward for 50cm.
            write_line('L.U-Turn-Step3', sign) #Save step 3.
        #-------------------------------
        elif move == "L.U-Turn-Step4":
            indicate_left()  #indicate left.
            gopigo3.left() #Turn left.
            time.sleep(2.2) #Sleep 2.2 seconds.
            gopigo3.stop() #Stop turning.
            write_line('L.U-Turn-Step4', sign) #Save step 4.
        #-------------------------------
        #-------------------------------
        elif move == "R.U-Turn-Step1":
            gopigo3.drive_cm(50) #drive forward for 50cm.
            write_line('R.U-Turn-Step1', sign) #Save step 1.
        #-------------------------------
        elif move == "R.U-Turn-Step2":
            indicate_right() #indicate right.
            gopigo3.right() #Turn right.
            time.sleep(2.2) #Sleep 2.2 seconds.
            gopigo3.stop() #Stop turning.
            write_line('R.U-Turn-Step2', sign) #Save step 2.
        #-------------------------------
        elif move == "R.U-Turn-Step3":
            gopigo3.drive_cm(50) #drive forward for 50cm.
            write_line('R.U-Turn-Step3', sign) #Save step 3.
        #-------------------------------
        elif move == "R.U-Turn-Step4":
            indicate_right() #indicate right.
            gopigo3.right() #Turn right.
            time.sleep(2.2) #Sleep 2.2 seconds.
            gopigo3.stop() #Stop turning.
            write_line('R.U-Turn-Step4', sign) #Save step 4.
        #-------------------------------            
#______________________________________________________________________   


print('________________________________________________')
print('***********-ROBOT MOVEMENTS ENABLED-***********')
print('           -ROBOT IS READY TO MOVE-            ')
print('________________________________________________')
print()


#----------------------------------------------------------------------
#BATTERY: Battery usage calculation...
#----------------------------------------------------------------------
def get_battery_usage():
    print()
    print()
    print()
    
    voltage = gopigo3.get_voltage_battery() #current voltage of the Robots battery.

    #calculate total current...
    #--------------------------------------
    motor_current = 0.75 * 2      #each  motor has 0.75A current.
    indication_lights = 0.02 * 2  #each light has 0.02A current.
    break_lights = 0.01 * 2       #each led lights has 0.01A current.

    total_current = motor_current + indication_lights + break_lights
    #--------------------------------------
    totBattery_capacity = 2  #my robot has 8 AA batteries each 2000mAh so total is 2Ah.

    #calculate Power Consumption...
    battery_life = totBattery_capacity / total_current 
    in_minutes = battery_life * 60

    battery_Duration = "Battery Life: Robot can operate for: " + str(round(in_minutes, 2)) + ' minutes.'

    print('___________________________________________________')
    print(battery_Duration)
    print('Current voltage:', str(voltage) + 'v')
    print('___________________________________________________')

    
get_battery_usage() #check battery status...
#----------------------------------------------------------------------



#=============================================================================




________________________________________________
***********-ROBOT MOVEMENTS ENABLED-***********
           -ROBOT IS READY TO MOVE-            
________________________________________________




___________________________________________________
Battery Life: Robot can operate for: 76.92 minutes.
Current voltage: 10.382v
___________________________________________________


In [4]:
""""
================================================================================================
**********************-GENERATE RANDOM AUTOMATED MOVEMENTS & SAVE TO CSV-***********************
================================================================================================

"""
#_______________________________________________________________
#GENERATE RANDOM AUTOMATED MOVEMENTS & SAVE TO CSV...
#_______________________________________________________________

#initialize the directions to pick from randomly...
lstDirections = ['forward', 'backward', 'left', 'right', 'left U-turn', 'right U-turn']

#----------------------------------------------------------------------
#FUNCTION: generates random directions...
#----------------------------------------------------------------------
def rand_direction(lstDirec, steps):
    direc = np.array([]) #initialize random direction arr...
    
    for i in range(steps): #number of loops = number of steps the robot is to take.
        randDirec = random.randint(0, len(lstDirections) - 1) #randomly pick from lstDirections directions to take.
        direc = np.append(direc, lstDirections[randDirec]) #append the random output to the arr of random directions.

    return direc #return the array of random directions.

#----------------------------------------------------
# Get the value from the widget
user_value = int_input.value 
#call the function to return the array of random directions.
arrDirec = rand_direction(lstDirections, user_value) 
#----------------------------------------------------
#----------------------------------------------------------------------  

#Save movements to a csv..
dirDirec = {'Saved directions' : arrDirec} #format as a dictionary before passing it to DataFrame.
dataFrame = pd.DataFrame(dirDirec) #Create DataFrame from the dictionary.
dataFrame.to_csv("saved_movements.csv", index=False) #save to CSV.
#----------------------------------------------------------------------


#----------------------------------------------------------------------
#FUNCTION: import directions...
#----------------------------------------------------------------------
def import_direc():
    dfImport = pd.read_csv("saved_movements.csv") #read csv...
    arrRetrieved = dfImport['Saved directions'].values #Populate arr with moves .
    return arrRetrieved #return retrieved moves. 

arrImportedDirec = import_direc() #call to retrieve moves from csv.
#----------------------------------------------------------------------
'''=============================================================================='''



""""
==============================================================================
**********************-REVERSE ALL MOVEMENTS BACKWARDS-***********************
==============================================================================

"""
#----------------------------------------------------------------------
#FUNCTION: Reverses the directions, first becomes the last direction. 
#----------------------------------------------------------------------
def reverse_array(arr):
    return arr[::-1] #return reversed version of the robot moves.

arrRevDirec = reverse_array(arrImportedDirec) #call to get reversed moves.

#----------------------------------------------------------------------
'''=============================================================================='''


""""
==============================================================================
***************************-CREATE CONTROL BUTTONS-***************************
==============================================================================

""" 

# Get the value from the widget
user_value = int_input.value
#print steps that the user chose...
print(f'User entered: {user_value}')



darkgrey = '#888888' #dark grey color code...
items_layout = Layout(flex='1 1 auto', width='auto') #config items layout.    

box_layout = Layout(display='flex', flex_flow='column', #config box button layout on display.
                    align_items='stretch', 
                    border='solid', width='30%')

def random_movement(_):
    #call function to run the retrieved moves in arr as unreversed moves ">>".
    move_robot(arrImportedDirec, '>> ') 
    
def reverse_movement(_):
    #call function to run the reversed moves in arr as reversed moves "<<".
    move_robot(arrRevDirec, '<< ') 
    
def stop_movement(_):
    stop_indicator() #call stop indicators.
    gopigo3.reset_all() #Reset all.

buttons = []
descriptions = ["Start Random Movements", "STOP", "Start Reversed Movements"] #Label for buttons.
callbacks = [random_movement, stop_movement, reverse_movement] #arr with button functions as callbacks.

for i in range(3):
    button = widgets.Button(description=descriptions[i], layout=items_layout) #Assign widget button to function.
    button.style.button_color = darkgrey if i != 1 else 'red' #Assign STOP button and make button Red.
    button.on_click(callbacks[i]) #On click activate functions on callbacks.
    buttons.append(button) #Append button on buttons.
    

display(widgets.VBox(buttons, layout=box_layout)) #Diplay buttons...
#----------------------------------------------------------------------
#========================================================================================

get_battery_usage() #check battery status...




User entered: 12





___________________________________________________
Battery Life: Robot can operate for: 76.92 minutes.
Current voltage: 10.665v
___________________________________________________


In [5]:
#_______________________________________________________________
#DISPLAY LIST OF THE MOVEMENTS...
#_______________________________________________________________
dataFrame.head(50) #Display movements...
#----------------------------------------------------------------------

,Saved directions
0,left
1,right U-turn
2,right
3,backward
4,left U-turn
5,backward
6,forward
7,backward
8,right U-turn
9,right


In [6]:
#_______________________________________________________________
#DISPLAY REVERSED LIST OF THE MOVEMENTS...
#_______________________________________________________________
dictReversed = {"Revesed directions": arrRevDirec} 
dfReversed = pd.DataFrame(dictReversed)
dfReversed.head(50)
#----------------------------------------------------------------------

,Revesed directions
0,backward
1,forward
2,right
3,right U-turn
4,backward
5,forward
6,backward
7,left U-turn
8,backward
9,right


In [7]:
""""
=======================================================================================
******************-RESTORE & RUN MOVEMENTS FROM LIVE MEMORY-***************************
=======================================================================================

""" 

#=================================================================================
import numpy as np
import pandas as pd
import random
import time

#__________________________________
#import Robot movements library...
#---------------------------
import easygopigo3 as easy
gopigo3 = easy.EasyGoPiGo3()
#---------------------------
#=================================================================================

#----------------------------------------------------------------------------
#FUNCTION: Check whether if there is content in the live memory text file...
#----------------------------------------------------------------------------
def check_content_exits():
    #----------------------------------------------------------------------
    #ERROR HANDLING: Check if live memory text file exist...
    #----------------------------------------------------------------------
    try:
        # Attempt to open a file in read mode
        file = open("live_memory.txt", "r")

    except FileNotFoundError:
        print("=====================================================")
        print("***-The file does not exist. Creating a new file.-***")
        print("=====================================================")
        print()
        # Create a new file
        file = open("live_memory.txt", "w")
        file.close()
    #----------------------------------------------------------------------    
    
    file = open("live_memory.txt", "r+")

    line = file.readline() #read line from live memory text file.

    if line == '': #check if there is content...
        print("====================================================================")
        print("*********-THERE IS NO AVAILABLE CONTENT IN THE LIVE MEMORY-*********")
        print('********************-ROBOT HAS NOT BEEN USED YET-********************')
        print("=====================================================================")
        
check_content_exits() #call to check if content exist.
#----------------------------------------------------------------------------

#----------------------------------------------------------------------------
#COLLECT MOVEMENTS: From CSV...
#----------------------------------------------------------------------------
csv_Direc = []
df = pd.read_csv("saved_movements.csv")
for move in df['Saved directions']:
    csv_Direc.append(move)
#----------------------------------------------------------------------------

#----------------------------------------------------------------------------
#COLLECT MOVEMENTS: From textFile...
#----------------------------------------------------------------------------
txt_moves = [] #initialize list.
with open('live_memory.txt','r+') as file: #open live memory for read and write.
    while True: 
        line = file.readline()
        if not line: #if reach end of file.
            break #break loop.
            
        readline = line.strip() #copy move to arr.
        txt_moves.append(readline) #append move in arr.
                    
file.close() #close live memory file.
#----------------------------------------------------------------------------


#---------------------------------------------------------------------------------
#FUNCTION: Clean values from text file...
#---------------------------------------------------------------------------------
def remove_signs(arrValue):
    sign = arrValue[:3] #slice arr to sign.
    arrValue = arrValue.replace(sign, '') #replace sign with empty string.
    arrValue = arrValue.replace('\n', '') #replace '\n' with empty string.
    return arrValue #return clean data of moves.

def clean_from_txt(arrTxt_moves):
    arrClean = [] #initialize list.
    for move in arrTxt_moves:
        arrClean.append(remove_signs(move)) #collect clean data of moves. 
    return arrClean #return populated arr with clean moves.
    #-----------------------------------------------------------------------------

clean_txt_moves = clean_from_txt(txt_moves) #collecte clean values.
#---------------------------------------------------------------------------------    


#---------------------------------------------------------------------------------
#FUCTION: Get full length of csv moves to compare to txt moves
#---------------------------------------------------------------------------------
def get_full_length(csv_Movements):
    csv_Length = len(csv_Movements) #get length of arr movements.
    
    for move in csv_Movements:
        if( move == 'left U-turn' or move == 'right U-turn'): #accommodate steps of U-turn.
            csv_Length = csv_Length + 3 #Add more length on length.
    return csv_Length #return length.
#---------------------------------------------------------------------------------


txt_moves_length = len(txt_moves)
csv_moves_length = get_full_length(csv_Direc)

if txt_moves_length == csv_moves_length: #txtFile and csvFile length are equal,
                            #Every moves have be perforemed by robot and saved.
    print()
    print("=================================================================")
    print("****-ALL MOVEMENTS ON LIVE MEMORY HAVE BEEN RAN SUCCESSFULLY-****")
    print("=================================================================")

elif txt_moves_length < csv_moves_length: #txtFile is less than csvFile length,
                         #Not all moves have be perforemed by robot and saved.
    #---------------------------------------------------------------------------------
    def activate_LiveMemory(ans, sign, txt_UnRan_Moves, csv_Direc):
        ans = ans.lower() #turn to lower case to avoid case sensitivity.
        
        if ans == 'no': #if users answer is No.
            print()
            print('___________________________________')
            print("Thanks for your reply, Enjoy :)")
            print('___________________________________')
            print()
        elif ans == 'yes': #if users answer is Yes.
            
            #---------------------------------------------------------------------------------
            #Make csv moves have the same format as txt file.
            #---------------------------------------------------------------------------------
            def format(csv_Direc):
                formated_csv = []
                for move in csv_Direc:
                    if move == 'left U-turn': #if left u-turn append left u-turn steps. 
                        formated_csv.append('L.U-Turn-Step1')
                        formated_csv.append('L.U-Turn-Step2')
                        formated_csv.append('L.U-Turn-Step3')
                        formated_csv.append('L.U-Turn-Step4')
                        
                    elif move == 'right U-turn': #if right u-turn append right u-turn steps. 
                        formated_csv.append('R.U-Turn-Step1')
                        formated_csv.append('R.U-Turn-Step2')
                        formated_csv.append('R.U-Turn-Step3')
                        formated_csv.append('R.U-Turn-Step4')
                    
                    else:
                        formated_csv.append(move) #if either append move anyway.
                return formated_csv
                #---------------------------------------------------------------------------------
            formated_csv = format(csv_Direc) #call csv movements to turn to live memory format.
            #---------------------------------------------------------------------------------
            
            #---------------------------------------------------------------------------------
            #FUNCTION: GET The unRan moves...
            #---------------------------------------------------------------------------------
            def remove_Ran_moves(txt_moves, formated_csv):
                txtlength = len(txt_moves) #get live memory move lengths
            
                csvlength = len(formated_csv) #get csv move lengths
            
                unRan_moves = formated_csv[txtlength : csvlength] 
            
                return unRan_moves #return moves...
                #---------------------------------------------------------------------------------

            print()
            print("______________________________________________")
            print("******-RUNNING LIVE MEMORY MOVEMENTS...-******")
            print("----------------------------------------------")
            
            if sign == '>>': #if live memory moves are in unreversed mode.
                unRan_moves = remove_Ran_moves(clean_txt_moves, formated_csv) #removes that have been ran already. 
                move_robot(unRan_moves, sign+' ') #Send to run on the 

            elif sign == '<<': #if live memory moves are in reversed mode.
                revFormated_csv = formated_csv[::-1] #Reverse moves before sending to run...
                unRan_moves = remove_Ran_moves(clean_txt_moves, revFormated_csv) #removes that have been ran already.    
                move_robot(unRan_moves, sign+' ') #Send to run on the  
            print("______________________________________________")

            
            print()
            print()
            print("============================================================================")
            print("****-ALL MOVEMENTS RESTORED FROM LIVE MEMORY HAVE BEEN RAN SUCCESSFULLY-****")
            print("============================================================================")
            #---------------------------------------------------------------------------------


                
    #---------------------------------------------------------------------------------
    
    file = open('live_memory.txt', 'r+') #open for read vand write.    
    
    line = file.readline() #read line. 
    
    sign =  line[:2] #get sign. 
    
    file.close() #close file.
    
    type_movement = sign #send to type of movements.
    #---------------------------------------------------------------------------------
    
    
    #---------------------------------------------------------------------------------
    #FUNCTIONS: Validate user input making sure the user inputs yes or no...
    #---------------------------------------------------------------------------------
    def validate_user_input(message):
        answer = input(message).lower() #turn to lower case.
        bool = False #set to false while user have not entered No or Yes.
        while bool == False: 
            if answer == 'yes': #if answer "yes".
                answer = answer
                bool = True #turn to true.
            elif answer == 'no':
                answer = answer
                bool = True #turn to true.
            else:
                print()
                print('_____________________________')
                print('REMEMBER: answer (yes/no)')
                print('-----------------------------')
                print()
                answer = input(message).lower() #turn to lower case.
                    
        return answer 
    #---------------------------------------------------------------------------------

    if type_movement == '>>': #if live memory moves are in unreversed mode.
        print('__________________________________________________')
        print("Last Ran movement(s) was during Random Movements.")
        print('----------------------------------------------------')

        answer = validate_user_input("Do you want to continue from last checkpoint (yes/no)?")       

        activate_LiveMemory(answer, type_movement, txt_moves, csv_Direc) #send answer to activate live memory function...
    
    elif type_movement == '<<': #if live memory moves are in reversed mode.
        print('____________________________________________________')
        print("Last Ran movement(s) was during Reversing Movements.")
        print('----------------------------------------------------')

        answer = validate_user_input("Do you want to continue from last checkpoint (yes/no)?")
            
        activate_LiveMemory(answer, type_movement, txt_moves, csv_Direc) #send answer to activate live memory function...
        
    #---------------------------------------------------------------------------------
    
#----------------------------------------------------------------------
#BATTERY: Battery usage calculation...
#----------------------------------------------------------------------
def get_battery_usage():
    print()
    print()
    print()
    
    voltage = gopigo3.get_voltage_battery() #current voltage of the Robots battery.

    #calculate total current...
    #--------------------------------------
    motor_current = 0.75 * 2      #each  motor has 0.75A current.
    indication_lights = 0.02 * 2  #each light has 0.02A current.
    break_lights = 0.01 * 2       #each led lights has 0.01A current.

    total_current = motor_current + indication_lights + break_lights
    #--------------------------------------
    totBattery_capacity = 2  #my robot has 8 AA batteries each 2000mAh so total is 2Ah.

    #calculate Power Consumption...
    battery_life = totBattery_capacity / total_current 
    in_minutes = battery_life * 60

    battery_Duration = "Battery Life: Robot can operate for: " + str(round(in_minutes, 2)) + ' minutes.'

    print('___________________________________________________')
    print(battery_Duration)
    print('Current voltage:', str(voltage) + 'v')
    print('___________________________________________________')

    
get_battery_usage() #check battery status...
#----------------------------------------------------------------------
    
#===============================================================================================



*********-THERE IS NO AVAILABLE CONTENT IN THE LIVE MEMORY-*********
********************-ROBOT HAS NOT BEEN USED YET-********************



___________________________________________________
Battery Life: Robot can operate for: 76.92 minutes.
Current voltage: 10.528v
___________________________________________________
